# Finance & Analytics Club

## CAGR (Compounded Annual Growth Rate)

CAGR is the annual rate of return realized by an asset/portfolio to reach its peak market value from its initial value.

CAGR calculation assumes that the profits are continuously reinvested.

It does not reflect investment rist therefore should always be used in conjunction with a volatility measure.

More at: https://www.investopedia.com/terms/c/cagr.asp

## Annualized Volatility

It is represented by the standard deviation of the returns. Captures the variability of return from the mean return.

It is a widely used method for risk, but does not capture tail risk.

More at: https://www.investopedia.com/terms/v/volatility.asp

## Sharpe Ratio

It is the average return earned in excess of the free risk rate per unit of volatility.

Sharpe ratio greater than 1 is good, greater than 2 is very good and greater than 3 is excellent.

Investors pay close attention to this metric while comparing funds.

More at: https://www.investopedia.com/terms/s/sharperatio.asp

## Sortino Ratio

It is a variation of Sharpe Ration which takes into account standard deviation of only negative returns.

Sharpe Ratio fails to determine the difference between the upward and downward fluctuations, Sortino makes that distinction and therefore considers only harmful volatility.

More at: https://www.investopedia.com/terms/s/sortinoratio.asp

## Maximum Drawdown & Calmar Ratio

Largest price drop in asset price over a specified time period is maximum dropdown.

Investments with longer backtesting period will likely have larger maximum dropdown and therefore caution must be applied in comparing across strategies.

Calmar Ratio is the ration between CAGR and maximum drawdown and it's measure of risk adjusted return.

More at:https://www.investopedia.com/terms/m/maximum-drawdown-mdd.asp, https://www.investopedia.com/terms/c/calmarratio.asp

Import necesary libraries

In [1]:
import pandas_datareader.data as pdr
import numpy as np
import datetime as dt

Download historical data for required stocks

In [2]:
ticker = "^GSPC"
SnP = pdr.get_data_yahoo(ticker,dt.date.today()-dt.timedelta(1825),dt.date.today())
SnP

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-06-29,2098.629883,2056.639893,2098.629883,2057.639893,3678960000,2057.639893
2015-06-30,2074.280029,2056.320068,2061.189941,2063.110107,4078540000,2063.110107
2015-07-01,2082.780029,2067.000000,2067.000000,2077.419922,3727260000,2077.419922
2015-07-02,2085.060059,2071.020020,2078.030029,2076.780029,2996540000,2076.780029
2015-07-06,2078.610107,2058.399902,2073.949951,2068.760010,3486360000,2068.760010
...,...,...,...,...,...,...
2020-06-19,3155.530029,3083.110107,3140.290039,3097.739990,8327780000,3097.739990
2020-06-22,3120.919922,3079.389893,3094.419922,3117.860107,4665380000,3117.860107
2020-06-23,3154.899902,3127.120117,3138.699951,3131.290039,4704830000,3131.290039


 ### Function to calculate the Cumulative Annual Growth Rate of a trading strategy

In [3]:
def CAGR(DF):
    df = DF.copy()
    df["daily_ret"] = DF["Adj Close"].pct_change()
    df["cum_return"] = (1 + df["daily_ret"]).cumprod()
    n = len(df)/252 # number of years, 252 is number of trading days in a year, use 52/12 if you have weekly/monthly data
    CAGR = (df["cum_return"][-1])**(1/n) - 1
    return CAGR

In [4]:
CAGR = CAGR(SnP)
CAGR

0.08442147700586444

### Function to calculate annualized volatility of a trading strategy

In [5]:
def volatility(DF):
    df = DF.copy()
    df["daily_ret"] = DF["Adj Close"].pct_change()
    vol = df["daily_ret"].std() * np.sqrt(252) # use 52/12 if you have weekly/monthly data
    return vol

In [6]:
vol = volatility(SnP)
vol

0.19335786799460325

### Function to calculate sharpe ratio ; rf is the risk free rate

In [7]:
def sharpe(DF,rf):
    sr = (CAGR - rf)/vol
    return sr

In [8]:
sharpe = sharpe(SnP, 0.063) # risk free rate in india is about 6.3%
sharpe

0.11078668392466104

### Function to calculate sortino ratio ; rf is the risk free rate

In [9]:
def sortino(DF,rf):
    df = DF.copy()
    df["daily_ret"] = DF["Adj Close"].pct_change()
    neg_vol = df[df["daily_ret"]<0]["daily_ret"].std() * np.sqrt(252)
    sr = (CAGR - rf)/neg_vol
    return sr

In [10]:
sortino  = sortino(SnP, 0.063) # risk free rate in india is about 6.3%
sortino

0.1234359874744036

### Function to calculate max drawdown

In [11]:
def max_dd(DF):
    df = DF.copy()
    df["daily_ret"] = DF["Adj Close"].pct_change()
    df["cum_return"] = (1 + df["daily_ret"]).cumprod()
    df["cum_roll_max"] = df["cum_return"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["cum_return"]
    df["drawdown_pct"] = df["drawdown"]/df["cum_roll_max"]
    max_dd = df["drawdown_pct"].max()
    return max_dd

In [12]:
mdd = max_dd(SnP)
mdd

0.33924960002653276

### Function to calculate calmar ratio

In [13]:
def calmar(DF):
    clmr = CAGR/mdd
    return clmr

In [14]:
CLMR = calmar(SnP)
CLMR

0.24884768323753906